# PFS 

The support for PFS files have been extended with MIKE IO release 1.2. It was previously only possible to *read* PFS files. It is now also possible to *modify* and *create* new PFS files. 

In [1]:
import mikeio

## Read

In [2]:
pfs = mikeio.read_pfs("../tests/testdata/pfs/lake.sw")
pfs

[FemEngineSW]
   [DOMAIN]
      Touched = 1
      discretization = 2
      number_of_dimensions = 2
      number_of_meshes = 1
      file_name = |.\Lake_Mesh.mesh|
      type_of_reordering = 1
      number_of_domains = 16
      coordinate_type = 'UTM-32'
      minimum_depth = 0.0
      datum_depth = 0.0
      vertical_mesh_type_overall = 1
      number_of_layers = 11
      z_sigma = 0.0
      vertical_mesh_type = 1
      layer_thickness = 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.0
      sigma_c = 0.0
      theta = 2.0
      b = 0.0
      number_of_layers_zlevel = 10
      vertical_mesh_type_zlevel = 1
      constant_layer_thickness_zlevel = 0.0
      variable_layer_thickness_zlevel = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
      type_of_bathymetry_adjustment = 1
      minimum_layer_thickness_zlevel = 0.0
      type_of_mesh = 0
      type_of_gauss = 3
      [BOUNDARY_NAMES]
         Touched = 0
         MzSEPfsListItemCount = 0
      EndSect  // BOUNDARY_NAMES
   End

The "target" (root section) is in this case called FemEngineSW. `pfs.FemEngineSW` is a PfsSection object that contains other PfsSection objects. Let's print the names of it's subsections:

In [3]:
pfs.FemEngineSW.keys()

dict_keys(['DOMAIN', 'TIME', 'MODULE_SELECTION', 'SPECTRAL_WAVE_MODULE'])

It is possible to navigate to each section and keyword in the pfs file:

In [4]:
pfs.FemEngineSW.DOMAIN.file_name

'|.\\Lake_Mesh.mesh|'

In [5]:
pfs.FemEngineSW.MODULE_SELECTION

Touched = 0
mode_of_hydrodynamic_module = 0
hydrodynamic_features = 1
fluid_property = 1
mode_of_spectral_wave_module = 2
mode_of_transport_module = 0
mode_of_mud_transport_module = 0
mode_of_eco_lab_module = 0
mode_of_sand_transport_module = 0
mode_of_particle_tracking_module = 0
mode_of_oil_spill_module = 0
mode_of_shoreline_module = 0

In [6]:
pfs.FemEngineSW.MODULE_SELECTION.mode_of_spectral_wave_module

2

If you are unsure the name of a section, it is also possible to search for a specific string in the file, to find the name of a specific section.

In the example below we do an case-insensitive search for the string 'charnock', which occurs at 6 different places in this file.

In [7]:
pfs.search("charnock")

[FemEngineSW]
   [SPECTRAL_WAVE_MODULE]
      [WIND]
         background_Charnock_parameter = 0.01
         Charnock_parameter = 0.01
      EndSect  // WIND
      [OUTPUTS]
         [OUTPUT_1]
            [MODEL_PARAMETERS]
               Charnock_constant = 0
            EndSect  // MODEL_PARAMETERS
         EndSect  // OUTPUT_1
         [OUTPUT_2]
            [MODEL_PARAMETERS]
               Charnock_constant = 0
            EndSect  // MODEL_PARAMETERS
         EndSect  // OUTPUT_2
         [OUTPUT_3]
            [MODEL_PARAMETERS]
               Charnock_constant = 0
            EndSect  // MODEL_PARAMETERS
         EndSect  // OUTPUT_3
         [OUTPUT_4]
            [MODEL_PARAMETERS]
               Charnock_constant = 0
            EndSect  // MODEL_PARAMETERS
         EndSect  // OUTPUT_4
      EndSect  // OUTPUTS
   EndSect  // SPECTRAL_WAVE_MODULE
EndSect  // FemEngineSW

The same search can be done at any level of the hierarchy, i.e. to search only within the OUTPUTS section:

In [8]:
pfs.FemEngineSW.SPECTRAL_WAVE_MODULE.OUTPUTS.search("charnock")

[OUTPUT_1]
   [MODEL_PARAMETERS]
      Charnock_constant = 0
   EndSect  // MODEL_PARAMETERS
EndSect  // OUTPUT_1
[OUTPUT_2]
   [MODEL_PARAMETERS]
      Charnock_constant = 0
   EndSect  // MODEL_PARAMETERS
EndSect  // OUTPUT_2
[OUTPUT_3]
   [MODEL_PARAMETERS]
      Charnock_constant = 0
   EndSect  // MODEL_PARAMETERS
EndSect  // OUTPUT_3
[OUTPUT_4]
   [MODEL_PARAMETERS]
      Charnock_constant = 0
   EndSect  // MODEL_PARAMETERS
EndSect  // OUTPUT_4

In [9]:
pfs.FemEngineSW.SPECTRAL_WAVE_MODULE.WIND.search("charnock")

background_Charnock_parameter = 0.01
Charnock_parameter = 0.01

MIKE FM PFS files has a specific structure and active FM modules can be accessed by an alias on the Pfs object. In this case, `pfs.SW` can be used as a short-hand for `pfs.FemEngineSW.SPECTRAL_WAVE_MODULE`.

In [10]:
pfs.SW.SPECTRAL.number_of_directions

16

In [11]:
pfs.SW.SPECTRAL.maximum_threshold_frequency

0.5959088268863615

Enumerated sections (e.g. [OUTPUT_1], [OUTPUT_2], ...) can be outputted in tabular form (dataframe).

In [12]:
df = pfs.SW.OUTPUTS.to_dataframe()
df

,Touched,include,title,file_name,type,format,flood_and_dry,coordinate_type,zone,input_file_name,...,last_time_step,time_step_frequency,number_of_points,POINT_1,LINE,AREA,INTEGRAL_WAVE_PARAMETERS,INPUT_PARAMETERS,MODEL_PARAMETERS,SPECTRAL_PARAMETERS
1,1,1,Wave parameters in domain,Wave_parameters.dfsu,1,2,2,UTM-32,0,||,...,450,10,1,"{'name': 'POINT_1', 'x': 20000.0, 'y': 20000.0}","{'npoints': 3, 'x_first': 0.0, 'y_first': 0.0,...","{'number_of_points': 4, 'POINT_1': {'x': -400....","{'Touched': 1, 'type_of_spectrum': 1, 'minimum...","{'Touched': 1, 'Surface_elevation': 0, 'Water_...","{'Touched': 1, 'Wind_friction_speed': 0, 'Roug...","{'Touched': 1, 'separation_of_wind_sea_and_swe..."
2,1,0,Wave parameters along line,Wave_line.dfs1,1,1,2,UTM-32,0,||,...,450,10,1,"{'name': 'POINT_1', 'x': 20000.0, 'y': 20000.0}","{'npoints': 41, 'x_first': 0.0, 'y_first': 200...","{'number_of_points': 4, 'POINT_1': {'x': -400....","{'Touched': 1, 'type_of_spectrum': 1, 'minimum...","{'Touched': 1, 'Surface_elevation': 0, 'Water_...","{'Touched': 1, 'Wind_friction_speed': 0, 'Roug...","{'Touched': 1, 'separation_of_wind_sea_and_swe..."
3,1,1,Wave parameters in a point,Waves_x20km_y20km.dfs0,1,0,2,UTM-32,0,||,...,450,1,1,"{'name': 'POINT_1', 'x': 38000.0, 'y': 20000.0}","{'npoints': 3, 'x_first': 0.0, 'y_first': 0.0,...","{'number_of_points': 4, 'POINT_1': {'x': -400....","{'Touched': 1, 'type_of_spectrum': 1, 'minimum...","{'Touched': 1, 'Surface_elevation': 0, 'Water_...","{'Touched': 1, 'Wind_friction_speed': 0, 'Roug...","{'Touched': 1, 'separation_of_wind_sea_and_swe..."
4,1,1,Spectrum in a point,spectrum_x20km_y20km.dfsu,4,0,2,UTM-32,0,||,...,450,10,1,"{'name': 'POINT_1', 'x': 38000.0, 'y': 20000.0}","{'npoints': 3, 'x_first': 0.0, 'y_first': 0.0,...","{'number_of_points': 4, 'POINT_1': {'x': -400....","{'Touched': 1, 'type_of_spectrum': 1, 'minimum...","{'Touched': 1, 'Surface_elevation': 0, 'Water_...","{'Touched': 1, 'Wind_friction_speed': 0, 'Roug...","{'Touched': 1, 'separation_of_wind_sea_and_swe..."


## Modify

The PfsSection object can be modified. Existing values can be changes, new key-value pairs can be added, subsections can added or removed. 

In [13]:
pfs.SW.SPECTRAL.number_of_directions = 32

In [14]:
pfs.SW.SPECTRAL

Touched = 1
type_of_frequency_discretization = 2
number_of_frequencies = 25
minimum_frequency = 0.055
frequency_interval = 0.02
frequency_factor = 1.1
type_of_directional_discretization = 1
number_of_directions = 32
minimum_direction = 0.0
maximum_direction = 0.0
separation_of_wind_sea_and_swell = 0
threshold_frequency = 0.125
maximum_threshold_frequency = 0.5959088268863615

### Add a new keyword

In [15]:
pfs.SW.SPECTRAL["new_keyword"] = "new_value"

In [16]:
pfs.SW.SPECTRAL

Touched = 1
type_of_frequency_discretization = 2
number_of_frequencies = 25
minimum_frequency = 0.055
frequency_interval = 0.02
frequency_factor = 1.1
type_of_directional_discretization = 1
number_of_directions = 32
minimum_direction = 0.0
maximum_direction = 0.0
separation_of_wind_sea_and_swell = 0
threshold_frequency = 0.125
maximum_threshold_frequency = 0.5959088268863615
new_keyword = 'new_value'

### Add a section

Let's create an additional output, by copying OUTPUT_4 and modifying some parameters.

In [17]:
pfs.SW.OUTPUTS.number_of_outputs += 1

In [18]:
new_output = pfs.SW.OUTPUTS.OUTPUT_4.copy()

In [19]:
new_output.file_name = 'spectrum_x10km_y40km.dfsu'
new_output.POINT_1.x = 10000
new_output.POINT_1.y = 40000

In [20]:
pfs.SW.OUTPUTS["OUTPUT_5"] = new_output

In [21]:
pfs.SW.OUTPUTS.keys()

dict_keys(['Touched', 'MzSEPfsListItemCount', 'number_of_outputs', 'OUTPUT_1', 'OUTPUT_2', 'OUTPUT_3', 'OUTPUT_4', 'OUTPUT_5'])

## Output

The Pfs object can be written to pfs file, but can also be exported to a dictionary (which in turn can be written to a yaml or json file).

In [22]:
pfs.write("lake_modified.pfs")

In [23]:
pfs.to_dict()

{'FemEngineSW': [DOMAIN]
    Touched = 1
    discretization = 2
    number_of_dimensions = 2
    number_of_meshes = 1
    file_name = |.\Lake_Mesh.mesh|
    type_of_reordering = 1
    number_of_domains = 16
    coordinate_type = 'UTM-32'
    minimum_depth = 0.0
    datum_depth = 0.0
    vertical_mesh_type_overall = 1
    number_of_layers = 11
    z_sigma = 0.0
    vertical_mesh_type = 1
    layer_thickness = 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.0
    sigma_c = 0.0
    theta = 2.0
    b = 0.0
    number_of_layers_zlevel = 10
    vertical_mesh_type_zlevel = 1
    constant_layer_thickness_zlevel = 0.0
    variable_layer_thickness_zlevel = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
    type_of_bathymetry_adjustment = 1
    minimum_layer_thickness_zlevel = 0.0
    type_of_mesh = 0
    type_of_gauss = 3
    [BOUNDARY_NAMES]
       Touched = 0
       MzSEPfsListItemCount = 0
    EndSect  // BOUNDARY_NAMES
 EndSect  // DOMAIN
 [TIME]
    Touched = 1
    start_time = 2002, 

In [24]:
# write to yaml file
import yaml
yaml.dump(pfs.to_dict(), open('lake_modified.yaml', 'w+'))

## Create

A PFS file can also be created from a dictionary, like this:

In [25]:
setup = {
    "Name": "Extract that",
    "InputFileName": "|random.dfs1|",
    "FirstTimeStep": 0,
    "LastTimeStep": 99,
    "X": 2,
    "OutputFileName": "|.\\out2.dfs0|",
}
t1_t0 = {"CLSID": "t1_t0.dll", "TypeName": "t1_t0", "Setup": setup}
t1_t0

{'CLSID': 't1_t0.dll',
 'TypeName': 't1_t0',
 'Setup': {'Name': 'Extract that',
  'InputFileName': '|random.dfs1|',
  'FirstTimeStep': 0,
  'LastTimeStep': 99,
  'X': 2,
  'OutputFileName': '|.\\out2.dfs0|'}}

In [26]:
pfs = mikeio.PfsDocument({"t1_t0": t1_t0})
pfs

[t1_t0]
   CLSID = 't1_t0.dll'
   TypeName = 't1_t0'
   [Setup]
      Name = 'Extract that'
      InputFileName = |random.dfs1|
      FirstTimeStep = 0
      LastTimeStep = 99
      X = 2
      OutputFileName = |.\out2.dfs0|
   EndSect  // Setup
EndSect  // t1_t0

In [27]:
pfs.write("extract_point.mzt")

## Clean up

In [28]:
import os
os.remove("lake_modified.pfs")
os.remove('lake_modified.yaml')
os.remove("extract_point.mzt")